In [1]:
import pandas as pd 
import numpy as np  
import matplotlib.pyplot as plt
import math
import datetime
from datetime import timedelta,date
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
from keras.models import Model, Sequential, load_model
from keras.callbacks import EarlyStopping
from keras.losses import MeanSquaredError, Huber,MeanAbsoluteError
from keras.layers import Dense, LSTM, Reshape,Dropout,Bidirectional
from keras.optimizers import Adam
from keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

from vmdpy import VMD

In [2]:
model_path = 'load_forecast.keras'
lstm_model = load_model(model_path)

In [3]:
list_years = [2021,2022]
aux_list = []
for year in list_years:
    df_read = pd.read_csv("CURVA_CARGA_"+str(year)+".csv", sep = ";")
    aux_list.append(df_read)
    
df_load = pd.concat(aux_list)

In [4]:
def vmd_timeseries(time_series_value):
    
    alpha = 2000     # moderate bandwidth constraint  
    tau = 0           # noise-tolerance (no strict fidelity enforcement)  
    k_nmodes = 40             # n modes  
    DC = 0             # no DC part imposed  
    init = 0           # initialize omegas uniformly  
    tol = 1e-6
    u, u_hat, omega = VMD(time_series_value,alpha,tau,k_nmodes,DC,init,tol)

    return(u)

In [5]:
def scale_back(scaled_values,val_min,val_max,label_columns,variable = 'windspeed'):

        if variable == 'windspeed':       
                min_values = val_min.tail(len(label_columns)).values
                max_values = val_max.tail(len(label_columns)).values
                
        elif variable == 'val_loss':
                min_values = val_min.head(1).values
                max_values = val_max.head(1).values

        scaled_back = scaled_values * (max_values - min_values) + min_values

        return scaled_back


In [6]:
def data_processing(df_load):

    df_load = df_load.rename(columns = {'din_instante': 'datetime',
                          'nom_subsistema':'sub',
                          'val_cargaenergiahomwmed':'carga_sub',
                          'id_subsistema':'id_sub'} )
    
    df_load = df_load.pivot_table(index = ["datetime"], columns=["id_sub"], values = "carga_sub")
    df_load.reset_index(inplace = True)

    #df_load.drop("id_sub", axis =1, inplace = True)

    df_load['datetime'] = pd.to_datetime(df_load['datetime'])

    #Summing all coluns to reach total load
    sub_markets =  df_load.select_dtypes(include=['float64', 'int64']).columns.to_list()
    df_load['total_load'] = df_load[sub_markets].sum(axis = 1)

    df_load.drop(columns = sub_markets, axis = 1, inplace = True)

    #Ciclical encoding datetime object:
    day = 1
    year = day*365

    #hourly data, each day has 24 timesteps 
    steps_per_day = 24

    #Convert into an number
    timestamp_s = df_load['datetime'].map(datetime.datetime.timestamp)

    #Ciclical enconding for sin and cos
    df_load = df_load.assign(year_sin = (np.sin(timestamp_s * (2*np.pi/year))).values)
    df_load = df_load.assign(year_cos = (np.cos(timestamp_s * (2*np.pi/year))).values)
    df_load = df_load.assign(day_sin = (np.sin(timestamp_s * (2*np.pi/day))).values)
    df_load = df_load.assign(day_cos = (np.cos(timestamp_s * (2*np.pi/day))).values)

    #Save the dates into another variable and remove it from the original dataframe
    saved_dates = df_load['datetime']
    saved_dates = saved_dates.reset_index(drop=True)

    df_load.drop('datetime', axis = 1, inplace = True)

    #Applying Variational Mode Decomposition into the wind speed values:
    #The objective is to decompose de original wind speed into K series
    #In which the sum of those K series are equal to the original wind speed value for each time step

    total_load = df_load['total_load'].values
    u = vmd_timeseries(total_load)

    #Iterate trought the decomposed wind speed array (u) and append its values to a column in the df_wind dataframe
    label_columns = []

    for i in range(len(u)):
        col_name = 'load_dec_' + str(i+1)
        df_load[col_name] = u[i]  
        label_columns.append('load_dec_'+str(i+1))

    #real_windspeed = df_wind['wind'].values

    df_load = df_load.drop(['total_load'],axis = 1)

    n = len(df_load)

    # Split 70:20:10 (train:validation:test)
    # train_df = df_load[0:int(n*0.99)]
    # val_df = df_load[int(n*0.7):int(n*0.9)]
    # test_df = df_load[int(n*0.9):]

    #define batch training size
    batch_size = 32

    #retrieve the max and min values of each feature. This will be used to bring those values back to the original
    #dimesion after scalling it to train the model
    min_values = df_load.min()
    max_values = df_load.max()

    #Scaling the dataframes to train
    scaler = MinMaxScaler()
    scaler.fit(df_load)

    scal_df = df_load.copy()
    scal_df[scal_df.columns] = scaler.transform(df_load[df_load.columns])

    return(scal_df,min_values,max_values)

In [7]:
df = data_processing(df_load)

In [ ]:
def model_validation()